In [ ]:
import numpy as np
from mlagents.tf_utils import tf
import os
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")
from animalai.envs.gym.environment import AnimalAIGym
import random
from animalai.envs.arena_config import ArenaConfig

import yaml
import copy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# CONFIG
env_path = '../env/macro2'
worker_id = 1
seed = 1
# yaml_path = '../examples/rand_env.yml'
ac = ArenaConfig("configurations/curriculum4/19.yml")
env = AnimalAIGym(environment_filename=env_path,
              worker_id=worker_id,
              n_arenas=1,
              arenas_configurations=ac,
                use_visual=False,
                seed=seed,
                 grayscale=False)
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph
# GRAPH_PB_PATH = './models/blindgotox9/AnimalAI/frozen_graph_def.pb'
GRAPH_PB_PATH = './macro_actions/go_behind.pb'
graph = load_pb(GRAPH_PB_PATH)

In [ ]:


import math
def transform(matrix, obj):
    """Transform world position of vector to local position relative to agent"""
    # obj = np.matrix(obj/40 - agent)
    obj.resize(4, refcheck=False) # adds 
    res  = matrix * obj.reshape(4,1)
    return res
def obs2state(vector_obs):
    obj_df = pd.DataFrame(vector_obs[6:146].reshape(-1,14))
    obj_df.rename(inplace=True, columns={0: 'id', 1:'class', 2: 'visible', 3:'abs_pos_x', 4:'abs_pos_y', 5:'abs_pos_z', 6: 'rel_pos_x', 7: 'rel_pos_y', 8:'rel_pos_z', 9:'abs_rot', 10:'rel_rot', 11:'size_x', 12:'size_y', 13:'size_z'})
    obj_df['class'] = obj_df['class'].apply(lambda x: obj_mapper[int(x)] if x!=-1.0 else 'null')
    obj_df = obj_df[obj_df['id']!=-1]
    obj_df['id'] = (obj_df['id']*-1).astype(int)
    obj_df = obj_df.set_index('id')

    # columns={0: 'vel_x', 1:'vel_z', 2:'rot_y', 3:'abs_pos_x', 4:'abs_pos_y', 5:'abs_pos_z'})

    rays = vector_obs[146:151].flatten()
    wtlm = np.matrix(vector_obs[151:167].flatten()).reshape(4,4)

    res = {
        "agent": vector_obs[0:6], #df
        "objects": obj_df, #df
        "rays": rays, # np
        "transform": wtlm, # np
    }
    return res
obj_mapper = [
             "Cardbox1",
             "Cardbox2",
             "CylinderTunnelTransparent",
             "CylinderTunnel",
             "DeathZone",
             "GoodGoalMulti",
             "GoodGoal",
             "LObject",
             "LObject2",
             "Ramp",
             "UObject",
             "WallTransparent",
             "Wall",
              "BadGoal",
              "HotZone"]
def preprocess(step_results):
    vector_obs = step_results[3]['batched_step_result'].obs[1].flatten()
    tmin1 = obs2state(vector_obs[:167])
    t = obs2state(vector_obs[167:])

    res = {
        "t-1": tmin1, #df
        "t": t, #df
        "reward": step_results[1], #float
        "done": step_results[2], # bool
        "step": step_results[-1] 
    }
    return res
def go_behind(step_results, x=None):
    """Go behind object x. x is an id. x comes in as "x,y"""
#     x = int(x.split(',')[0])
    res = np.zeros(20)
    mstate = preprocess(step_results)
    x = mstate['t']['objects']
    x = x[x['class']=='GoodGoal'].index[0]
    for c, timestep in enumerate(['t-1', 't']):
        state = mstate[timestep]
        ooi_pos = np.array(state['objects'].loc[x, ['abs_pos_x', 'abs_pos_y', 'abs_pos_z']])
        agent_pos = state['agent'][3:6]
        rel_pos = ooi_pos-agent_pos
        magn = np.linalg.norm(rel_pos)
        behind_pos = rel_pos*(magn+0.3)/magn # 0.125 => 5units in world
        local_behind_pos = transform(state['transform'], behind_pos)
        res[0+(10*c):2+(10*c)] = state['agent'][:2]
        res[2+(10*c)] = state['objects'].loc[x,'rel_rot']
#         res[3+(10*c):5+(10*c)] = [local_behind_pos[0], local_behind_pos[2]] #only want x,z
        res[3+(10*c):5+(10*c)] = [state['objects'].loc[x,'rel_pos_x'],
                                 state['objects'].loc[x,'rel_pos_z']] #only want x,z
        res[5+(10*c):10+(10*c)] = state['rays']
    return res



In [ ]:

sr = env.step([[0,0]])

while not sr[3]['batched_step_result'].done:
    with tf.compat.v1.Session(graph=graph) as sess:
        output_node = graph.get_tensor_by_name('policy_1/concat_2:0')
        input_node = graph.get_tensor_by_name('vector_observation:0')
        action_masks = graph.get_tensor_by_name('action_masks:0')
        state = go_behind(sr)
        model_result = sess.run(
            output_node,
            feed_dict ={
            input_node : np.array(
                [state]),
                 action_masks: np.array([[1,1,1,1,1,1]])})
    action = [model_result[0,:3].argmax(), model_result[0, 3:].argmax()]
    sr = env.step([action])
print(sr[1])

In [ ]:
mstate = preprocess(sr)
x = mstate['t']['objects']
display(x)
x = x[x['class']!='GoodGoal'].index[0]

In [ ]:
x